# Preparing Data
Load the training and testing data from the **.npy** file (NumPy array).

In [1]:
import numpy as np

print('Loading data ...')

data_root = '../data/timit/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


# Create Dataset

In [2]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self,X,y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None
    def __gititem__(self,index):
        if self.label is not None:
            return self.data[index],self.label[index]
        else:
            return self.data[index]
    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set,
you can modeify the  variable **VAL_RATIO** to change the ratio of validation data 

In [3]:
VAL_RATIO =0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))         #计算训练集数量
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]     #划分训练集和验证集
print("Size of training set : {}".format(train_x.shape))
print("Size of validation set : {}".format(val_x.shape))

Size of training set : (983945, 429)
Size of validation set : (245987, 429)


Create a dataloader  from the dataset,  

In [4]:
BATCH_SIZE = 64
from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x,val_y)
train_loader = DataLoader(train_set,batch_size= BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=False)    #验证集不用进行打乱


Cleanup the unneeded variables to save memory.

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [5]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

21126

# Create Model

定义模型结构

In [7]:
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(429,1024),
            nn.Sigmoid(),
            nn.Linear(1024,512),
            nn.Sigmoid(),
            nn.Linear(512,128),
            nn.Sigmoid(),
            nn.Linear(128,39)
        )
    def forward(self):
        return self.net()

# Training

In [9]:
def get_device():
    return 'cuda:0' if torch.cuda.is_available() else 'cpu'

固定随机种子以提高可重复性。

In [10]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

训练参数

In [14]:
same_seeds(0)

device = get_device()
print(f'DEVICE:{device}')

num_epoch = 20              
learning_rate = 0.0001      #学习率

model_path = '../data/timit/model.ckpt'

model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


DEVICE:cpu


training

In [ ]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader):
        x, y = data
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        batch_loss = criterion(outputs,y)
        _, train_pred = torch.max(outputs,1)
        batch_loss.backward()
        optimizer.step()

        train_acc += (train_pred.cpu() == y.cpu()).sum().item()
        train_loss += batch_loss

    # validition
    if len(val_set)>0:
        model.eval()
        with torch.no_grad():
            for i ,data in enumerate(val_loader):
                x, y = data
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                batch_loss = criterion(outputs,y)
                _, val_pred = torch.max(outputs,1)

                val_acc += (val_pred.cpu() == y.cpu()).sum().item()
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train ACC : {:3.6f}  Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(epoch+1, num_epoch, train_acc/len(train_set), val_acc/len(val_set), )) 

# if not validating, save the last epoch
if len(val_set) == 0 :
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

# Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = Dataloader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint

model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))



make prediction

In [ ]:
predict=[]
model.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader):
        